In [1]:
%pip install numpy
%pip install seaborn

import os
import re
import json
import glob
import math
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import plotly.express as px
import plotly.io as pio

pd.set_option("display.max_colwidth", 200)
pd.set_option("display.width", 160)
pd.set_option("display.max_columns", 200)

sns.set_theme(style="whitegrid", context="talk")
plt.rcParams["figure.figsize"] = (12, 6)

# Rohstring
verzeichnis = r"Untitled Folder\Untitled Folder\records_raw.csv"

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.2.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.2.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


ModuleNotFoundError: No module named 'plotly'

In [ ]:
# ----------- 1) Dateien einsammeln und Name parsen -----------------
import re
verzeichnis = "Untitled Folder/Untitled Folder"
name_re = re.compile(r'^(aws|azure|gcp)_(?:([A-Za-z0-9-]+)_)?(\d+)_(cold|warm)_results_(\d+)\.json$', re.IGNORECASE)

def parse_context(base):
    m = name_re.match(base)
    if not m:
        return None
    provider = m.group(1).lower()
    region = m.group(2)  # kann None sein
    workload = int(m.group(3))
    phase = m.group(4).lower()
    use_case = int(m.group(5))
    return provider, region, workload, phase, use_case

dateien = [f for f in os.listdir(verzeichnis) if f.lower().endswith('.json')]
matches = [base for base in dateien if parse_context(base) is not None]
print(f"Gefundene JSONs im Ordner: {len(dateien)}")
print(f"Dateien passend zum Schema: {len(matches)}")
print(matches[:10])
if not matches:
    raise SystemExit("Keine passenden Dateien gefunden. Prüfe Ordnernamen oder Dateinamen.")

NameError: name 're' is not defined

In [ ]:
# Pfad, in dem die JSONs liegen (ggf. anpassen)
verzeichnis = "Desktop/Untitled Folder/Untitled Folder"
files = [f for f in os.listdir(verzeichnis) if f.lower().endswith('.json')]
matches = [base for base in dateien if parse_context(base) is not None]

# Manifest aufbereiten
rows = []
pat = re.compile(r"_((?:warm)|(?:cold))_results_(\d+)\.json$")
for fn in files:
    m = pat.search(os.path.basename(fn))
    phase = m.group(1) if m else ""
    memory_mb = int(m.group(2)) if m else None
    rows.append({"filename": fn, "phase": phase, "memory_mb": memory_mb})

manifest = pd.DataFrame(rows).sort_values(["phase","memory_mb"]).reset_index(drop=True)
display(manifest)

# Persistieren, damit nachfolgende Schritte einen stabilen Einstiegspunkt haben
manifest_path = os.path.join(verzeichnis, "input_manifest.csv")
manifest.to_csv(manifest_path, index=False)
manifest_path

In [ ]:
def _safe_get(d, path, default=None):
    cur = d
    for k in path:
        if isinstance(cur, dict) and k in cur:
            cur = cur[k]
        else:
            return default
    return cur

def parse_sebs_file(path, phase, memory_mb):
    """
    Liest eine SEBS-JSON-Datei und gibt eine Liste flacher Records (dict) zurück.
    Die Funktion ist robust gegenüber fehlenden Feldern.
    """
    with open(path, "r", encoding="utf-8") as f:
        data = json.load(f)
    
    invocations_root = data.get("_invocations", {})
    records = []
    
    # Jede Funktion (hier: ein Key wie 'sebs_..._uploader_python_3_11_x64')
    for func_name, func_invocations in invocations_root.items():
        # Jede Invocation (Key ist request_id)
        for req_id, payload in func_invocations.items():
            # times
            times = payload.get("times", {}) or {}
            t_client = times.get("client")
            t_benchmark = times.get("benchmark")
            t_http_first_byte = times.get("http_first_byte_return")
            t_http_startup = times.get("http_startup")
            t_initialization = times.get("initialization")

            # output/result/measurement
            output = payload.get("output", {}) or {}
            is_cold_flag = output.get("is_cold")
            result = output.get("result", {}) or {}
            measurement = result.get("measurement", {}) or {}
            compute_time = measurement.get("compute_time")
            upload_time = measurement.get("upload_time")
            download_time = measurement.get("download_time")

            # stats
            stats = payload.get("stats", {}) or {}
            cold_start_stats = stats.get("cold_start")

            # begin/end from output
            begin_ts = output.get("begin")
            end_ts = output.get("end")
            
            rec = {
                "function": func_name,
                "request_id": req_id,
                "phase": phase,
                "memory_mb": memory_mb,
                "is_cold_flag": is_cold_flag,
                "cold_start_stats": cold_start_stats,
                "begin_ts": begin_ts,
                "end_ts": end_ts,
                "compute_time": compute_time,
                "upload_time": upload_time,
                "download_time": download_time,
                "client_time": t_client,
                "benchmark_time": t_benchmark,
                "http_first_byte_return": t_http_first_byte,
                "http_startup": t_http_startup,
                "initialization_time": t_initialization,
                # Roh-JSON Pfad für Repro
                "source_file": os.path.basename(path),
            }
            records.append(rec)
    return records

In [ ]:
from pathlib import Path

missing = []
all_records = []
for _, row in manifest.iterrows():
    fn = Path(verzeichnis) / row["filename"]
    if not fn.exists():
        missing.append(str(fn))
        continue
    all_records.extend(
        parse_sebs_file(str(fn), phase=row["phase"], memory_mb=int(row["memory_mb"]))
    )

if missing:
    print("Fehlende Dateien:\n- " + "\n- ".join(missing))


In [ ]:
# Manifest laden (stabiler Einstiegspunkt)
manifest = pd.read_csv(os.path.join(verzeichnis, "input_manifest.csv"))



In [ ]:

all_records = []
for _, row in manifest.iterrows():
    fn = os.path.join(verzeichnis, row["filename"])
    recs = parse_sebs_file(fn, phase=row["phase"], memory_mb=int(row["memory_mb"]))
    all_records.extend(recs)


In [ ]:
df_raw = pd.DataFrame(all_records)
display(df_raw.head(10))
print(f"Rows: {len(df_raw):,}")

# Persistieren (Rohdaten)
raw_csv_path = os.path.join(verzeichnis, "records_raw.csv")
df_raw.to_csv(raw_csv_path, index=False)
raw_csv_path


In [ ]:
df = df_raw.copy()

# Numerische Felder sauber konvertieren
num_cols = [
    "compute_time", "upload_time", "download_time",
    "client_time", "benchmark_time", "http_first_byte_return",
    "http_startup", "initialization_time"
]
for c in num_cols:
    df[c] = pd.to_numeric(df[c], errors="coerce")

# Mikrosekunden -> Millisekunden
for c in num_cols:
    df[c + "_ms"] = df[c] / 1000.0

# IO-Teil und Gesamtdauer
df["io_time_ms"] = df["upload_time_ms"].fillna(0) + df["download_time_ms"].fillna(0)

# "Echte" Cold/Warm pro Invocation
# - is_cold_flag kommt aus output.is_cold (True/False)
# - cold_start_stats kommt aus stats.cold_start (True/False)
# Wir priorisieren output.is_cold, fallback auf stats.cold_start, sonst Phase-Heuristik.
df["is_cold"] = np.where(
    df["is_cold_flag"].notna(), df["is_cold_flag"],
    np.where(df["cold_start_stats"].notna(), df["cold_start_stats"], df["phase"].eq("cold"))
).astype(bool)

# Handliche Spaltennamen
df = df.rename(columns={
    "client_time_ms": "client_ms",
    "compute_time_ms": "compute_ms",
    "upload_time_ms": "upload_ms",
    "download_time_ms": "download_ms",
    "benchmark_time_ms": "benchmark_ms",
})

keep_cols = [
    "function", "request_id", "phase", "memory_mb", "is_cold",
    "client_ms", "compute_ms", "upload_ms", "download_ms",
    "io_time_ms", "benchmark_ms", "http_first_byte_return_ms",
    "http_startup_ms", "initialization_time_ms",
    "begin_ts", "end_ts", "source_file"
]
df = df[[c for c in keep_cols if c in df.columns]]

display(df.head(10))


In [ ]:
print(df.groupby(["phase","memory_mb","is_cold"])["request_id"].count())
print()
print("Fehlende Kombinationen:", df.groupby(["phase","memory_mb"])["request_id"].count().unstack(fill_value=0).eq(0).any().any())

In [ ]:
def percentiles(s):
    return pd.Series({
        "count": s.count(),
        "mean": s.mean(),
        "std": s.std(),
        "p50": s.quantile(0.50),
        "p90": s.quantile(0.90),
        "p99": s.quantile(0.99),
        "min": s.min(),
        "max": s.max(),
    })

group_cols = ["phase", "memory_mb"]

agg_client = df.groupby(group_cols)["client_ms"].apply(percentiles).unstack()
agg_compute = df.groupby(group_cols)["compute_ms"].apply(percentiles).unstack()
agg_io = df.groupby(group_cols)["io_time_ms"].apply(percentiles).unstack()

display(agg_client.round(3))
display(agg_compute.round(3))
display(agg_io.round(3))

# Persistieren
out_dir = verzeichnis
agg_client.to_csv(os.path.join(out_dir, "summary_client_ms.csv"))
agg_compute.to_csv(os.path.join(out_dir, "summary_compute_ms.csv"))
agg_io.to_csv(os.path.join(out_dir, "summary_io_ms.csv"))


In [ ]:
# 1) Median End-to-End (client_ms) vs. Memory nach Phase
med = df.groupby(["phase","memory_mb"])["client_ms"].median().reset_index()
fig = sns.lineplot(data=med, x="memory_mb", y="client_ms", hue="phase", marker="o")
plt.title("Median Client-Zeit (ms) pro Memory und Phase")
plt.xlabel("Speicher (MB)")
plt.ylabel("Client-Zeit (ms)")
plt.legend(title="Phase")
plt.tight_layout()
plt.show()

# 2) Boxplot: Verteilung Client-Zeit je Memory und Phase
fig = sns.boxplot(data=df, x="memory_mb", y="client_ms", hue="phase", showfliers=False)
plt.title("Client-Zeit (ms) – Verteilung nach Memory und Phase")
plt.xlabel("Speicher (MB)")
plt.ylabel("Client-Zeit (ms)")
plt.legend(title="Phase")
plt.tight_layout()
plt.show()

# 3) Gestapelte Balken: Mittelwerte Compute vs. IO je Gruppe
mean_parts = (
    df.groupby(["phase","memory_mb"])
      .agg(mean_compute_ms=("compute_ms","mean"),
           mean_io_ms=("io_time_ms","mean"))
      .reset_index()
)
mean_parts_melt = mean_parts.melt(id_vars=["phase","memory_mb"], value_vars=["mean_compute_ms","mean_io_ms"],
                                  var_name="komponente", value_name="ms")
fig = sns.barplot(data=mean_parts_melt, x="memory_mb", y="ms", hue="komponente")
plt.title("Durchschnitt Compute vs. IO (ms) pro Memory und Phase")
plt.xlabel("Speicher (MB)")
plt.ylabel("Zeit (ms)")
plt.legend(title="Komponente")
plt.tight_layout()
plt.show()


In [ ]:
tidy_path = os.path.join(verzeichnis, "records_tidy.csv")
df.to_csv(tidy_path, index=False)
tidy_path



In [ ]:
# Basis-Kennzahlen
cols = ["client_ms", "benchmark_ms", "compute_ms", "io_time_ms", "upload_ms", "download_ms"]
summary = df.groupby("is_cold")[cols].agg(["count","median","mean","std","min","max"])
display(summary)

# Speicher-sensitiv
mem_summary = df.groupby(["memory_mb","is_cold"])[cols].median().reset_index()
display(mem_summary)

# Per Function + Phase
fn_phase = df.groupby(["function","phase"])[cols].quantile([0.5,0.95]).unstack()
display(fn_phase)
